### Import Libraries

In [ ]:
import pandas as pd
import numpy as np 
from tqdm import tqdm
from tensorflow.keras.preprocessing import image
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D,MaxPool2D,Dense,Flatten,BatchNormalization,Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.regularizers import l2

In [ ]:
# Read train csv file
poster_df = pd.read_csv("train.csv")

### Feature Engineeing

In [ ]:
# Convert values in genre to list
poster_df['Genre'] = poster_df['Genre'].apply(eval)

poster_df = poster_df.explode('Genre')

# One Hot Encode genre file
df_encoded = pd.get_dummies(poster_df, columns=['Genre'])

df_final = df_encoded.groupby('Id').sum().reset_index()

### Image Processing

In [ ]:
width = 350
height = 350
x = []
for i in tqdm(range(len(df_final))):
    path = 'Movie_Poster_Dataset/' + df_final['Id'][i]+'.jpg'
    img = image.load_img(path , target_size = (width , height,3))
    img = image.img_to_array(img)
    img = img / 255.0
    x.append(img)

x = np.array(x) # independent features

In [ ]:
y = df_final.drop(['Id'] , axis=1) # dependent feautures
y = y.to_numpy()

In [ ]:
x_train , x_test , y_train , y_test = train_test_split(x , y , test_size= 0.2)

### Modelling

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=x_train[0].shape , kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(MaxPool2D(2,2))
model.add(Dropout(0.3))

model.add(Conv2D(64, kernel_size=(3,3), activation='relu', kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(MaxPool2D(2,2))
model.add(Dropout(0.3))

model.add(Conv2D(128, kernel_size=(3,3), activation='relu', kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(MaxPool2D(2,2))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(25, activation='sigmoid'))

model.summary()

In [ ]:
est = model.fit(x_train,y_train , epochs=5,validation_data=(x_test,y_test))

In [ ]:
model.save('genre_detection_model.h5')